In [16]:
from neo4j import GraphDatabase
from dotenv import load_dotenv
import os
import pandas as pd

In [17]:
load_dotenv()
URI = os.environ.get('NEO4J_URI')
USERNAME = os.environ.get('NEO4J_USERNAME')
PASSWORD = os.environ.get('NEO4J_PASSWORD')
DB_PATH= os.environ.get('NEO4J_DB_PATH')

In [18]:
driver = GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))

In [19]:
def run_query(query, params={}):
    with driver.session() as session:
        result = session.run(query, params)
        return result.data()  # Returns list of dictionaries


In [22]:
import shutil
import os

source = os.getcwd().replace("\\", "/") + "/csv/affiliation.csv"
destination = DB_PATH + "/import"

shutil.copy(source, destination) 

'C:\\Users\\josub\\Documents\\Neo4j\\relate-data\\dbmss\\dbms-63530268-3389-4bdc-9b02-07e7210fa2f0/import\\affiliation.csv'

In [ ]:
file="csv/reviews.csv"
df=pd.read_csv(file, sep="|")
bad_rev=df[df["type"]=="bad"]["review"].to_list()
good_rev=df[df["type"]=="good"]["review"].to_list()


#I use modulos instead of rand() to make it reproducible (Seeds are not supported)

queries=["""
LOAD CSV FROM 'file:///affiliation.csv' AS row
FIELDTERMINATOR '|'
MERGE (n:affiliation {affiliationId: toInteger(row[0]), name: row[1], type: row[2]})
""",
"""
MATCH (a:author)
MATCH (af:affiliation)
WITH a, COLLECT(af) as affs, SIZE(COLLECT(af)) as total
WITH a, affs[ID(a) % total] as m
MERGE (a)-[:affiliated_to]->(m)
""",
"""
MATCH (n:journal|event)
SET n.reviewerNumber = CASE
    WHEN ID(n)%3 = 0 THEN 1 
    WHEN ID(n)%3 = 1 THEN 3
    ELSE 5
END
""",
"""
MATCH (j:journal)-[:published]-(p:paper)-[c:reviewed_by]-(a:author)
WITH j, p, collect(c) AS list
WHERE j.reviewerNumber = 1
DELETE  list[1],list[2]

UNION 

MATCH (j:journal)-[:published]-(p:paper)-[c:reviewed_by]-(a:author)
WHERE j.reviewerNumber = 5
MATCH (au:author)
WHERE NOT (au)-[:writes|reviewed_by]-(p:paper)
WITH p, collect(au)[0..2] as authors
UNWIND authors AS auth
MERGE (p)-[:reviewed_by]->(auth)
""",
"""
MATCH (j:journal)-[:published]-(p:paper)-[c:reviewed_by]-(a:author)
SET c.resolution="Rejected"

UNION 

MATCH (j:journal)-[:published]-(p:paper)-[c:reviewed_by]-(a:author)
WHERE j.reviewerNumber = 1
SET c.resolution="Accepted"

UNION

MATCH (j:journal)-[:published]-(p:paper)-[c:reviewed_by]-(a:author)
WHERE j.reviewerNumber = 3
WITH p, collect(c)[0..((ID(p)%2)+2)] as approved
UNWIND approved AS appr
SET appr.resolution="Accepted"

UNION

MATCH (j:journal)-[:published]-(p:paper)-[c:reviewed_by]-(a:author)
WHERE j.reviewerNumber = 5
WITH p, collect(c)[0..((ID(p)%3)+3)] as approved
UNWIND approved AS appr
SET appr.resolution="Accepted"
""",
f"""
MATCH (:paper)-[r:reviewed_by]-(:author)
WITH r, {bad_rev} AS bad_rev, SIZE({bad_rev}) AS bad_total, {good_rev} AS good_rev, SIZE({good_rev}) AS good_total
SET r.review = CASE
    WHEN r.resolution="Accepted" THEN good_rev[ID(r) % good_total]
    ELSE bad_rev[ID(r) % bad_total]
END
"""
]

for query in queries:
    run_query(query)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 5, column: 14, offset: 110} for query: '\nMATCH (a:author)\nMATCH (af:affiliation)\nWITH a, COLLECT(af) as affs, SIZE(COLLECT(af)) as total\nWITH a, affs[ID(a) % total] as m\nMERGE (a)-[:affiliated_to]->(m)\n'


In [ ]:
#CHECK QUERIES
query3="""
MATCH (j:journal)-[:published]-(p: paper)-[c:reviewed_by]-(a:author)
WITH j, p, COUNT(c) AS cnt
WHERE j.reviewerNumber<>cnt
RETURN j.reviewerNumber, cnt
"""
query4="""
MATCH (j:journal)-[:published]-(p: paper)-[c:reviewed_by]-(a:author)
WHERE c.resolution="Accepted"
WITH j, p, COUNT(c) AS cnt
RETURN j.reviewerNumber, cnt
"""


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 4, column: 10, offset: 62} for query: '\nMATCH (n:journal|event)\nSET n.reviewerNumber = CASE\n    WHEN ID(n)%3 = 0 THEN 1 \n    WHEN ID(n)%3 = 1 THEN 3\n    ELSE 5\nEND\n'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 5, column: 10, offset: 91} for query: '\nMATCH (n:journal|event)\nSET n.reviewerNumber = CASE\n    WHEN ID(n)%3 = 0 THEN 1 \n    WHEN ID(n)%3 = 1 THEN 3\n    ELSE 5\nEND\n'


TransientError: {code: Neo.TransientError.General.MemoryPoolOutOfMemoryError} {message: The allocation of an extra 1,6 GiB would use more than the limit 1,4 GiB. Currently using 619,1 MiB. dbms.memory.transaction.total.max threshold reached}